# Angular Inflation with Anguelova

In [2]:
import sympy
import numpy as np
from einsteinpy.symbolic import MetricTensor
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

#import inflatox
import inflatox

In [3]:
sympy.init_printing()
#setup the coordinates
p, x = sympy.symbols("phi chi")
coords = [p, x]
d = len(coords)

#setup the potential
mp, mx, a = sympy.symbols("m_phi m_chi alpha")
V = a/2 * ( (mp*p)**2 + (mx*x)**2 ).nsimplify()

#setup the metric
metric_diagonal = 6*a / (1 - p**2 - x**2)**2
metric = [[0 for i in range(2)] for i in range(2)]
metric[0][0] = metric_diagonal
metric[1][1] = metric_diagonal
metric = MetricTensor(metric, coords)

#display the metric and potential
display(metric.tensor())
display(V)

⎡      6⋅α                         ⎤
⎢────────────────         0        ⎥
⎢               2                  ⎥
⎢⎛   2    2    ⎞                   ⎥
⎢⎝- χ  - φ  + 1⎠                   ⎥
⎢                                  ⎥
⎢                        6⋅α       ⎥
⎢       0          ────────────────⎥
⎢                                 2⎥
⎢                  ⎛   2    2    ⎞ ⎥
⎣                  ⎝- χ  - φ  + 1⎠ ⎦

In [4]:
hesse = inflatox.SymbolicCalculation.new(coords, metric, V).execute([[0,1]])

Calculating orthonormal basis...


<IPython.core.display.Math object>

TypeError: cannot determine truth value of Relational